In [1]:
import sys
sys.path.append('..')
import json
import help_functions as hf
import numpy as np
from collections import Counter
from matplotlib import pyplot as plt
import pandas as pd
hf.setup_gpu(gpu_nr=1)

2 Physical GPUs, 1 Logical GPU


In [2]:
i = 11
with open('training_configurations.json', 'r') as fp:
    config = json.load(fp)[str(i)]
print(config)

{'epochs': 15, 'augment': False, 'basemodel': 'EfficientNetB2', 'nr_classes': 20, 'batch_size': 32, 'undersample': False, 'oversample': True, 'data_folder': 'data/split_dataframes_heuristic_labels_20221006', 'loss_function': 'binary_crossentropy', 'class_weights': False, 'image_dimension': 64, 'random_initialization': False, 'number_trainable_layers': 0, 'results_folder': 'results_thesis/11_oversample__bs_32'}


In [3]:
train, train_df = hf.get_flow(df_file=config['data_folder'] + '/test_df.json.bz2',
                    batch_size=config['batch_size'],
                    nr_classes=config['nr_classes'],
                    image_dimension=config['image_dimension'])

y_true = hf.get_y_true(shape=(train.samples, len(train.class_indices)), classes=train.classes)


Found 50000 non-validated image filenames belonging to 40 classes.


/home/matvieir/wiki_image_classification/src/classification/help_functions.py:271: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x_labels['labels'] = df['labels'].apply(lambda labels_list: [label for label in labels_list if label in top_classes])


Found 49514 validated image filenames belonging to 20 classes.


In [17]:
y_true.shape

(49514, 20)

In [4]:
original_nr_rows = y_true.shape[0]
nr_labels = y_true.shape[1]
y_true_copy = np.copy(y_true)
indices_to_add = []

BIG_NUMBER = 10_000
label_rewards = BIG_NUMBER / y_true_copy.sum(axis=0)
row_rewards = (label_rewards * y_true_copy).sum(axis=1)

reps = 20
while y_true_copy.shape[0] < original_nr_rows * (1 + 0.2):

    best_row = y_true_copy[np.argmax(row_rewards), :].reshape(1, nr_labels)
    y_true_copy = np.append(y_true_copy, np.tile(best_row, (reps, 1)), axis=0)

    label_rewards = BIG_NUMBER / y_true_copy.sum(axis=0)
    row_rewards = (label_rewards * y_true_copy).sum(axis=1)

    idx_to_add = np.where((y_true == best_row).all(axis=1))[0]
    for _ in range(reps):
        indices_to_add.append(idx_to_add)


In [9]:
indices_to_add

[array([37441]),
 array([37441]),
 array([37441]),
 array([37441]),
 array([37441]),
 array([37441]),
 array([37441]),
 array([37441]),
 array([37441]),
 array([37441]),
 array([37441]),
 array([37441]),
 array([37441]),
 array([37441]),
 array([37441]),
 array([37441]),
 array([37441]),
 array([37441]),
 array([37441]),
 array([37441]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]),
 array([42620]

In [10]:
indices_to_add_hashable = [tuple([el[0]]) for el in indices_to_add]


In [12]:
duplicate_indices_dict = dict(Counter(indices_to_add_hashable))

In [13]:
# duplicate_indices_dict = hf.oversample(y_true, 
#                                         list(train.class_indices.keys()), 
#                                         0.2, 
#                                         config['results_folder'])
print('found indices to duplicate...')
for index_to_duplicate in duplicate_indices_dict:
    times_to_duplicate = duplicate_indices_dict[index_to_duplicate]
    train_df = pd.concat([train_df, pd.DataFrame([train_df.iloc[index_to_duplicate]] * times_to_duplicate)], axis=0, ignore_index=True)
print('LOG: or here')
train_df = train_df.reset_index() # not necessary but why not
print('LOG: starting to get new flow... does it fail here?')
train, _ = hf.get_flow(df=train_df,
                        nr_classes=config['nr_classes'],
                        image_dimension=config['image_dimension'])

found indices to duplicate...
LOG: or here
LOG: starting to get new flow... does it fail here?
Found 59434 non-validated image filenames belonging to 20 classes.
Found 59434 validated image filenames belonging to 20 classes.
